# Example 3: Galaxy-galaxy lensing 

In this example, we'll show how to calculate $\Delta\Sigma(r),$ the galaxy-galaxy lensing signal of a mock catalog.  

In [1]:
from halotools.empirical_models import PrebuiltSubhaloModelFactory
model = PrebuiltSubhaloModelFactory('behroozi10')

from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(simname='chinchilla', redshift=0.2, halo_finder='rockstar', version_name = 'Lb400-2048',\
                           dz_tol = 0.01, ptcl_version_name = 'Lb400-2048')
#model.populate_mock(halocat)

#imports
from glob import glob
from os import path
import numpy as np
from halotools.sim_manager import CachedHaloCatalog, RockstarHlistReader, UserSuppliedPtclCatalog
from helpers.readGadgetSnapshot import readGadgetSnapshot
from AbundanceMatching import *

In [2]:
#time to trim
galaxy_magnitudes = np.genfromtxt('gama_matched_catalog.npy')
downsample_idxs = np.genfromtxt('gama_matched_catalog_idxs.npy')

In [3]:
downsample_idxs = downsample_idxs.astype(int)

In [4]:
#do the downsampling 
galaxy_magnitudes = galaxy_magnitudes[downsample_idxs]

Predictions for galaxy-galaxy lensing are calculated from the cross-correlation between the galaxy positions and the dark matter density field. For the latter, we will use the dark patter particle positions stored in the `ptcl_table` attribute of the mock.

In [6]:
px = halocat.ptcl_table['x']
py = halocat.ptcl_table['y']
pz = halocat.ptcl_table['z']

In [8]:
#px = ptclcat.ptcl_table['x']
#py = ptclcat.ptcl_table['y']
#pz = ptclcat.ptcl_table['z']
import numpy as np
from halotools.mock_observables import return_xyz_formatted_array

Nptcls_to_keep = int(5e4)
randomizer = np.random.random(len(px))
sorted_randoms = np.sort(randomizer)
ptcl_mask = np.where(sorted_randoms < sorted_randoms[Nptcls_to_keep])[0]
particle_positions = return_xyz_formatted_array(px, py, pz, mask = ptcl_mask)

Now we will extract the *x, y, z* positions of various subsamples of our galaxies 

In [ ]:
#take them from our SHAM, not the mock
magnitude_cut1 = galaxy_magnitudes < -19
magnitude_cut2 = galaxy_magnitudes < -20
magnitude_cut3 = galaxy_magnitudes < -21
magnitude_cut4 = galaxy_magnitudes < -22

x1 = halocat.halo_table['halo_x'][downsample_idxs][magnitude_cut1]
y1 = halocat.halo_table['halo_y'][downsample_idxs][magnitude_cut1]
z1 = halocat.halo_table['halo_z'][downsample_idxs][magnitude_cut1]

x2 = halocat.halo_table['halo_x'][downsample_idxs][magnitude_cut2]
y2 = halocat.halo_table['halo_y'][downsample_idxs][magnitude_cut2]
z2 = halocat.halo_table['halo_z'][downsample_idxs][magnitude_cut2]

x3 = halocat.halo_table['halo_x'][downsample_idxs][magnitude_cut3]
y3 = halocat.halo_table['halo_y'][downsample_idxs][magnitude_cut3]
z3 = halocat.halo_table['halo_z'][downsample_idxs][magnitude_cut3]

x4 = halocat.halo_table['halo_x'][downsample_idxs][magnitude_cut4]
y4 = halocat.halo_table['halo_y'][downsample_idxs][magnitude_cut4]
z4 = halocat.halo_table['halo_z'][downsample_idxs][magnitude_cut4]

In [ ]:
#turn it into an array
all_positions1 = return_xyz_formatted_array(x1,y1,z1)
all_positions2 = return_xyz_formatted_array(x2,y2,z2)
all_positions3 = return_xyz_formatted_array(x3,y3,z3)
all_positions4 = return_xyz_formatted_array(x4,y4,z4)

In [ ]:
from halotools.mock_observables import delta_sigma

rp_bins = np.logspace(-1,1,15)

particle_masses = halocat.particle_mass
period=halocat.Lbox[0]
#downsampling_factor = (halocat.num_ptcl_per_dim**3)/float(len(particle_positions))
downsampling_factor = Nptcls_to_keep

In [ ]:
from astropy import cosmology
cosmo =  cosmology.core.LambdaCDM(H0=100 * 0.7, Om0=0.286, Ode0=0.714)

In [ ]:
rp, result_all_in_mpc1 = delta_sigma(all_positions1, particle_positions, 
                                     particle_masses, downsampling_factor,
                                    rp_bins, period, cosmology= cosmo, num_threads='max')
rp, result_all_in_mpc2 = delta_sigma(all_positions2, particle_positions, 
                                     particle_masses, downsampling_factor,
                                    rp_bins, period, cosmology= cosmo, num_threads='max')
rp, result_all_in_mpc3 = delta_sigma(all_positions3, particle_positions, 
                                     particle_masses, downsampling_factor,
                                    rp_bins, period, cosmology= cosmo, num_threads='max')
rp, result_all_in_mpc4 = delta_sigma(all_positions4, particle_positions, 
                                     particle_masses, downsampling_factor,
                                    rp_bins, period, cosmology= cosmo, num_threads='max')

In [ ]:
length_unit_conversion = 1e6
result_all_in_pc1 = result_all_in_mpc1/length_unit_conversion**2
result_all_in_pc2 = result_all_in_mpc2/length_unit_conversion**2
result_all_in_pc3 = result_all_in_mpc3/length_unit_conversion**2
result_all_in_pc4 = result_all_in_mpc4/length_unit_conversion**2

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#plot it
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

__=plt.loglog()

__=ax.plot(rp, result_all_in_pc1, label=r'All galaxies magnitudes < -19')
__=ax.plot(rp, result_all_in_pc2, label=r'All galaxies magnitudes < -20')
__=ax.plot(rp, result_all_in_pc3, label=r'All galaxies magnitudes < -21')
__=ax.plot(rp, result_all_in_pc4, label=r'All galaxies magnitudes < -22')

__=ax.set_xlim(xmin = 0.1, xmax = 10)
__=ax.set_ylim(ymin = 0.5, ymax = 200)

__=ax.set_xlabel(r'$R_{\rm p} $  $\rm{[Mpc / h]}$', fontsize=16)
__=ax.set_ylabel(r'$\Delta\Sigma(R_{\rm p})$  $[h M_{\odot} / {\rm pc}^2]$', fontsize=16)
__=ax.legend(loc='best', fontsize=13)
__=plt.xticks(fontsize=15); plt.yticks(fontsize=15) 
